In [1]:
def ObjectView(Obj,**Kwargs):
    if type(Obj) not in {tuple,list,dict,set,frozenset}: return ('"'+Temp+'"') if TypeTemp == str else str(Temp)
    class Dummy:
        def __init__(Self,Text=""):
            Self.__Text = Text
        def __str__(Self): return Self.__Text
    TypeMap={tuple:"tuple",list:"list",dict:"dict",set:"set",frozenset:"frozenset"}
    ThemeMap = dict((
    ("tuple",{"startB0":"=","startB1":"=","start0":"=","start1":"=","startB>":"╔","start>":"╦",
    "continue":"║","add":"╠","end":"╚"}),
    ("list",{"startB0":"~","startB1":"~","start0":"~","start1":"~","startB>":"╓","start>":"╥",
    "continue":"║","add":"╟","end":"╙"}),
    ("dict",{"startB0":"╫","startB1":"╫","start0":"╫","start1":"╫","startB>":"╓","start>":"╥","startI":"╥",
    "continue":"║","add":"╟","end":"╙","endI":"╰"}),
    ("set",{"startB0":"┼","startB1":"┼","start0":"┼","start1":"┼","startB>":"┌","start>":"┬",
    "continue":"│","add":"├","end":"└"}),
    ("frozenset",{"startB0":"╪","startB1":"╪","start0":"╪","start1":"╪","startB>":"╒","start>":"╤",
    "continue":"│","add":"╞","end":"╘"})))
    # although python dict support objects as keys, strings are used for interoperability
    ThemeMapDefault = " "
    if "theme" in Kwargs and Kwargs["theme"] != None:
        for _k,_p in Kwargs["theme"].items():
            if type(_k) == str and _k in {"tuple","list","dict","set","frozenset"}:
                for __k,__p in _p.items():
                    ThemeMap[_k][__k] = __p
    for _v in ("tuple","list","dict","set","frozenset"):
        ThemeMap[ThemeMap[_v]["startB>"]]=ThemeMap[_v]["continue"]
        ThemeMap[ThemeMap[_v]["start>"]]=ThemeMap[_v]["continue"]
        ThemeMap[ThemeMap[_v]["continue"]]=ThemeMap[_v]["continue"]
        ThemeMap[ThemeMap[_v]["add"]]=ThemeMap[_v]["continue"]
        if _v == "dict": ThemeMap[ThemeMap[_v]["startI"]]=ThemeMap[_v]["continue"]
    AjoinSet = set()
    for _v in ("tuple","list","dict","set","frozenset"):
        AjoinSet.add(ThemeMap[_v]["startB>"])
        AjoinSet.add(ThemeMap[_v]["start>"])
        AjoinSet.add(ThemeMap[_v]["continue"])
        AjoinSet.add(ThemeMap[_v]["add"])
        if _v == "dict": AjoinSet.add(ThemeMap[_v]["startI"])
    Res = [""]
    Types = [None,]
    Lens = [1,]
    Leaves = []
    Queue = [iter((Obj,))]
    while True:
        Temp = next(Queue[-1])
        Lens[-1] -= 1
        TypeTemp = type(Temp)
        if TypeTemp in {tuple,list,dict,set,frozenset}:
            if len(Temp) == 0:
                Types.append(type(Dummy))
                Lens.append(1)
                Queue.append(iter((Dummy(ThemeMap[TypeMap[TypeTemp]]["start"+("B" if len(Queue) == 1 else "")+"0"]),)))
                continue
            Types.append(TypeTemp)
            Lens.append(len(Temp) * (2 if TypeTemp == dict else 1))
            if TypeTemp == dict:
                Res[-1] += ThemeMap[TypeMap[TypeTemp]]["start"+("B" if len(Queue) == 1 else "")\
                +("1" if Lens[-1] == 2 else ">")]+ThemeMap[TypeMap[TypeTemp]]["startI"]
            else:
                Res[-1] += ThemeMap[TypeMap[TypeTemp]]["start"+("B" if len(Queue) == 1 else "")+("1" if Lens[-1] == 1 else ">")]
            Queue.append(iter((_c for _r in Temp.items() for _c in _r) if TypeTemp == dict else Temp))
        else:
            NewRows = len(Leaves)
            Leaves.extend((('"'+Temp+'"') if TypeTemp == str else str(Temp)).splitlines())
            NewRows = len(Leaves) - NewRows
            AddRow = "".join(ThemeMap.get(_v,ThemeMapDefault) for _v in Res[-1])
            Res.extend((AddRow,)*(NewRows-1))
            while Lens[-1] == 0:
                Lens.pop()
                Types.pop()
                Queue.pop()
                if len(Queue) == 1: return "\n".join(Branch+Leaf for Branch,Leaf in zip(Res,Leaves)) 
            FindTemp = len(Res[-1]) - 1
            while Res[-1][FindTemp] not in AjoinSet:
                # FindTemp will never reach below 0 as there is always a root
                FindTemp -= 1
            if Types[-1] == dict:
                Res.append(AddRow[:FindTemp]\
                            +(ThemeMap[TypeMap[Types[-1]]]["endI"] if Lens[-1] % 2 == 1\
                            else (ThemeMap[TypeMap[Types[-1]]]["end" if Lens[-1] == 2 else "add"]\
                            +ThemeMap[TypeMap[Types[-1]]]["startI"])))
            elif Types[-1] in {tuple,list,set,frozenset}:
                Res.append(AddRow[:FindTemp]+ThemeMap[TypeMap[Types[-1]]]["end" if Lens[-1] == 1 else "add"])

In [5]:
Val = [["x",["y",[(["z"],)]]],{"K0":"P0",(("K1",),):"P1",(("u",("v",)),):["aaa",{"000":"111"},"bbb"]},["A","B","C"],["a",(9,(8,7)),"c"],1,2,3,((4,),)]
print("Result:\n"+ObjectView(Val))
Val2 = {(((),),):{-0.0:[[[-0.0,]]]},frozenset((frozenset((frozenset(),)),)):set((frozenset(),)),None:{"Not\nbad\nat\nall":([("A\njob\nwell\ndone",)],),False:abs,True:dict()}}
print("Result:\n"+ObjectView(Val2))
Theme = {"tuple":{"startB0":"═","start0":"═","startB1":"═","start1":"═"},
         "list":{"startB0":"─","startB1":"─","start0":"─","start1":"─"},
         "dict":{"startI":"╥","endI":"╙"}}
print("Theme:\n"+ObjectView(Theme))
print("Result using a theme:\n"+ObjectView(Val2,theme=Theme))

Result:
╓╥"x"
║╙╥"y"
║ ╙~=~"z"
╟╥╥"K0"
║║╰"P0"
║╟╥=="K1"
║║╰"P1"
║╙╥=╦"u"
║ ║ ╚="v"
║ ╰╥"aaa"
║  ╟╫╥"000"
║  ║ ╰"111"
║  ╙"bbb"
╟╥"A"
║╟"B"
║╙"C"
╟╥"a"
║╟╦9
║║╚╦8
║║ ╚7
║╙"c"
╟1
╟2
╟3
╙==4
Result:
╓╥===
║╰╫╥-0.0
║  ╰~~~-0.0
╟╥╪╪╪
║╰┼╪
╙╥None
 ╰╥╥"Not
  ║║bad
  ║║at
  ║║all"
  ║╰=~="A
  ║    job
  ║    well
  ║    done"
  ╟╥False
  ║╰<built-in function abs>
  ╙╥True
   ╰╫
Theme:
╓╥"tuple"
║╰╥╥"startB0"
║ ║╰"═"
║ ╟╥"start0"
║ ║╰"═"
║ ╟╥"startB1"
║ ║╰"═"
║ ╙╥"start1"
║  ╰"═"
╟╥"list"
║╰╥╥"startB0"
║ ║╰"─"
║ ╟╥"startB1"
║ ║╰"─"
║ ╟╥"start0"
║ ║╰"─"
║ ╙╥"start1"
║  ╰"─"
╙╥"dict"
 ╰╥╥"startI"
  ║╰"╥"
  ╙╥"endI"
   ╰"╙"
Result using a theme:
╓╥═══
║╙╫╥-0.0
║  ╙───-0.0
╟╥╪╪╪
║╙┼╪
╙╥None
 ╙╥╥"Not
  ║║bad
  ║║at
  ║║all"
  ║╙═─═"A
  ║    job
  ║    well
  ║    done"
  ╟╥False
  ║╙<built-in function abs>
  ╙╥True
   ╙╫
